In [ ]:
import torch

from src.core.eqprop import nn as enn

input_size = 5
output_size = 2

x = torch.rand(1, input_size, requires_grad=True)
y = torch.rand(1, output_size)
layer = enn.EqPropLinear(input_size, output_size, bias=False)
optimizer = torch.optim.SGD(layer.parameters(), lr=layer.solver.beta)

In [ ]:
import torch.nn.functional as F

print(layer.weight)
logit = layer(x)
y_hat = F.softmax(logit, dim=1)
loss = F.mse_loss(y_hat, y)
optimizer.zero_grad()
loss.backward()
print(layer.weight.grad)
print(x.grad)
optimizer.step()
print(layer.weight)